In [1]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"


In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


In [3]:
import tkinter as tk
from tkinter import scrolledtext
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import requests
import wikipedia
import wolframalpha

In [ ]:
# Load Hugging Face Chatbot Model
model_name = "facebook/blenderbot-400M-distill"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [ ]:
# API Keys (Replace with your keys)
WOLFRAM_API_KEY = "2QAQGW-XGQ65KE4KP"
WEATHER_API_KEY = "fc904dd228fe02d3838ff46e02cc69e0"
NEWS_API_KEY = "44d42e5d5c0049bfacf25335183cfd7b"

In [ ]:
# WolframAlpha Client
wolfram_client = wolframalpha.Client(WOLFRAM_API_KEY)

In [ ]:
def get_wikipedia_answer(query):
    try:
        query = query.replace("who is", "").replace("what is", "").strip()
        result = wikipedia.summary(query, sentences=5)  # Increased to 5 sentences
        return result
    except wikipedia.exceptions.DisambiguationError as e:
        return f"Your query is too broad. Try searching for: {', '.join(e.options[:3])}"
    except wikipedia.exceptions.PageError:
        return "I couldn't find an answer on Wikipedia."
    except Exception as e:
        return f"Error retrieving Wikipedia data: {str(e)}"


In [ ]:
def get_wolfram_answer(query):
    try:
        query = query.replace("calculate", "").replace("solve", "").strip()
        res = wolfram_client.query(query)
        pods = [pod.text for pod in res.pods if pod.text]

        if pods:
            return " ".join(pods[:3])  # Take the first 3 meaningful responses
        else:
            return "I couldn't solve that. Try rephrasing your question."
    except:
        return "I couldn't solve that. Try rephrasing your question."


In [ ]:
weather_api_key = "fc904dd228fe02d3838ff46e02cc69e0"

def get_weather(city):
    try:
        city = city.replace("weather in", "").strip()
        url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={weather_api_key}&units=metric"
        response = requests.get(url).json()
        
        if response.get("cod") != 200:
            return f"Sorry, I couldn't find the weather for {city}."

        temperature = response["main"]["temp"]
        description = response["weather"][0]["description"]
        return f"The temperature in {city} is {temperature}°C with {description}."
    except Exception as e:
        return f"Error retrieving weather data: {str(e)}"


In [ ]:
# News Function
def get_news():
    try:
        url = f"https://newsapi.org/v2/top-headlines?country=us&apiKey={NEWS_API_KEY}"
        response = requests.get(url).json()
        articles = response["articles"][:5]
        return "\n".join([f"🔹 {article['title']}" for article in articles])
    except:
        return "Couldn't fetch news."

In [ ]:
def chatbot_response(user_input):
    user_input_lower = user_input.lower()

    if "who is" in user_input_lower or "what is" in user_input_lower:
        return get_wikipedia_answer(user_input)
    elif "calculate" in user_input_lower or "solve" in user_input_lower:
        return get_wolfram_answer(user_input)
    elif "weather in" in user_input_lower:
        city = user_input_lower.replace("weather in", "").strip()
        return get_weather(city)
    elif "latest news" in user_input_lower:
        return get_news()
    else:
        # Tokenize user input
        inputs = tokenizer(user_input, return_tensors="pt", padding=True, truncation=True)

        # Generate response with increased length
        response_ids = model.generate(
            inputs["input_ids"], 
            attention_mask=inputs["attention_mask"],  
            max_length=300,  # Increased max_length for longer responses
            pad_token_id=tokenizer.eos_token_id
        )
        
        return tokenizer.decode(response_ids[0], skip_special_tokens=True)


In [ ]:
# GUI Setup
def send_message(event=None):
    user_text = user_input.get()
    if user_text.strip():
        chat_display.config(state=tk.NORMAL)
        chat_display.insert(tk.END, f"🧑 You: {user_text}\n", "user")
        chat_display.config(state=tk.DISABLED)

        response = chatbot_response(user_text)

        chat_display.config(state=tk.NORMAL)
        chat_display.insert(tk.END, f"🤖 Chatbot: {response}\n", "bot")
        chat_display.config(state=tk.DISABLED)

        chat_display.yview(tk.END)
        user_input.delete(0, tk.END)

In [ ]:
# GUI Window
root = tk.Tk()
root.title("AI Chatbot")
root.geometry("500x650")
root.configure(bg="#222831")


In [ ]:
# Chat Display
chat_display = scrolledtext.ScrolledText(root, state=tk.DISABLED, wrap=tk.WORD, bg="#393E46", fg="white", font=("Arial", 12))
chat_display.pack(pady=10, padx=10, expand=True, fill=tk.BOTH)

In [ ]:
# Input Frame
input_frame = tk.Frame(root, bg="#222831")
input_frame.pack(pady=5, padx=10, fill=tk.X)

In [ ]:
# Input Field
user_input = tk.Entry(input_frame, width=50, font=("Arial", 12), bg="#EEEEEE", fg="black")
user_input.pack(side=tk.LEFT, expand=True, fill=tk.X, padx=5, pady=5)
user_input.bind("<Return>", send_message)


In [ ]:
# Send Button
send_button = tk.Button(input_frame, text="Send", command=send_message, font=("Arial", 12), bg="#00ADB5", fg="white", padx=10)
send_button.pack(side=tk.RIGHT, padx=5, pady=5)

In [ ]:
# Run GUI
root.mainloop()